In [ ]:
%matplotlib widget
from sympy import *
init_printing(use_latex=True)

# 15 - Implementing an Equation class

## 15.3 - Using the Equation class

In [ ]:
from sympy_utils import Equation

In [ ]:
t, k, x, m, Cp, rho, alpha = symbols(r"t, k, x, m, C_p, rho, alpha", positive=True)
Dt, Dx, Dy, Dz, DV = [symbols(r"{\Delta}" + s, positive=True) for s in "t,x,y,z,V".split(",")]
Q, q, P, T = [e(t, x) for e in symbols("Q, q, P, T", cls=Function)]

In [ ]:
Mass = Equation(rho, m / DV)
Veq = Equation(DV, Dx * Dy * Dz)
Qeq = Equation(Q, m * Cp * T)
fourier = Equation(q, -k * T.diff(x))
td = Equation(alpha, k / (Cp * rho))
display(Mass, Veq, Qeq, fourier, td)

In [ ]:
# change in heat content
hc_change = Q - Q.subs(t, t + Dt)
# heat in: Dy * Dz is the surface area through which q flows
hi = q * Dt * Dy * Dz
# heat out: Dy * Dz is the surface area through which q flows
ho = q.subs(x, x + Dx) * Dt * Dy * Dz
# heat generated
hg = P * Dt * Dx * Dy * Dz
# heat balance equation
hb = Equation(hc_change, hi - ho + hg)
hb

In [ ]:
Veq = Equation(DV, Dx * Dy * Dz)
hb = hb.dorhs(lambda e: ((e * Dx).expand().subs(*Veq.reversed.args) / Dx)).expand()
hb

In [ ]:
hb = (hb / Dt).expand()
hb

In [ ]:
hb = hb.dolhs(lambda e: Limit(e.collect(1 / Dt), Dt, 0))
hb

In [ ]:
hb = hb.dolhs(lambda e: e.subs(e, Q.diff(t)))
hb

In [ ]:
qxdx = Equation(q.subs(x, x + Dx), q.subs(x, x + Dx).series(Dx, 0, 2).removeO().doit())
qxdx

In [ ]:
hb = hb.subs(qxdx).expand()
hb

In [ ]:
hb = hb.subs(fourier, side="right").subs(Qeq, side="left").doit()
hb

In [ ]:
hb = (hb / DV / k).expand().subs(Mass.reversed, side="left")
hb

In [ ]:
hb = hb.subs(1 / td.reversed, side="left")
hb